## Modelling

In [ ]:
import numpy as np

X, y = dftenant_reviews[neg_flags].copy(), dftenant_reviews.review.copy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)


from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y) # Data leakage?
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

print([x.shape for x in (X_train, X_test, y_train, y_test)])

print("after stratified: classes in train and test:", np.unique(y_train), np.unique(y_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


clf = RandomForestClassifier(random_state=42)

param_grid = [
    dict(
        criterion = ["gini", "entropy"], 
        #splitter = ["best", "random"], 
        max_depth = [None,1,2,4,6,8], 
        min_samples_split = [10, 16, 20], 
        min_samples_leaf = [5,10,15], 
        # min_weight_fraction_leaf = 0, 
        max_features = ["sqrt", "log2", ], 
        # random_state = [42], 
        max_leaf_nodes = [None,2,4,8,16], 
        min_impurity_decrease = [0, 0.05, 0.10], 
        # class_weight = None, 
        # ccp_alpha = 0
    )
]


np.prod([len(p) for p in param_grid[0].values()])


In [ ]:
# gs = GridSearchCV(clf,param_grid, scoring='accuracy', n_jobs=4, verbose=3).fit(X_train,y_train)

In [ ]:
dfgsres = pd.DataFrame(gs.cv_results_).sort_values('rank_test_score').reset_index(drop=True).set_index('params')
dfgsres[['mean_test_score', 'std_test_score', 'rank_test_score']].head(50)


In [ ]:
clf.set_params(**{'criterion': 'entropy', 'max_depth': 4, 'max_features': 'log2', 'max_leaf_nodes': 8, 'min_impurity_decrease': 0, 'min_samples_leaf': 5, 'min_samples_split': 16, 'splitter': 'random'})

clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

names = ['train', 'test']
Xs = [X_train, X_test]
ys = [y_train, y_test]

metrics = {}

for name, X, y in zip(names, Xs, ys):

    metrics[name] = dict(
        accuracy_score=accuracy_score(y, clf.predict(X)),
        roc_auc_score=roc_auc_score(y, clf.predict_proba(X), multi_class="ovr")
    )

display(pd.DataFrame(metrics))


In [ ]:
from sklearn.tree import plot_tree

# fig,ax= plt.subplots(figsize=(40,10))
# _ = plot_tree(clf, fontsize=9, filled=True, feature_names=clf.feature_names_in_, class_names=True)
